In [12]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API

class UpworkAnalyzer:
    
    def __init__(self):
        # Write a log
        self.log = open('log_upwork_data_analysis_2017_11_08_worldwide_allskills_trial.txt', 'a')
        self.log.write("We have started analyzing data!" + "\n")

        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Get detailed_info from workers in our database
        self.cur.execute("SELECT detailed_info FROM upwork_worldwide_allskills_2017_10_21 LIMIT 3;")
        
        # Initialize arrays for Causal Analysis 
        self.user_count = 0
        self.bill_rate_array = []
        self.gender_array = []
        self.all_covariates_array = []
        
    def identify_gender(self):
        for user in self.cur:
            photo = user[0]["dev_portrait_50"]
            print photo
            
        self.recognize_faces()
        
    def recognize_faces(self):
        ###############################################
        #### Update or verify the following values. ###
        ###############################################

        # Replace the subscription_key string value with your valid subscription key.
        subscription_key = '1fad9b94ae3d44fcaa17a6848f06d086' # This is key 1

        # Replace or verify the region.
        #
        # You must use the same region in your REST API call as you used to obtain your subscription keys.
        # For example, if you obtained your subscription keys from the westus region, replace 
        # "westcentralus" in the URI below with "westus".
        #
        # NOTE: Free trial subscription keys are generated in the westcentralus region, so if you are using
        # a free trial subscription key, you should not need to change this region.
        uri_base = 'westcentralus.api.cognitive.microsoft.com'

        # Request headers.
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': subscription_key,
        }

        # Request parameters.
        params = urllib.urlencode({
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'false',
            'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
        })

        # The URL of a JPEG image to analyze.
        beg_body = "{'url':"
        end_body = "}"
        path = "https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/images_trial/eureka_1.jpeg"
        body = beg_body + '"' + path + '"' + end_body 
        print body
        
        try:
            # Execute the REST API call and get the response.
            conn = httplib.HTTPSConnection('westcentralus.api.cognitive.microsoft.com')
            conn.request("POST", "/face/v1.0/detect?%s" % params, body, headers)
            response = conn.getresponse()
            data = response.read()

            # 'data' contains the JSON data. The following formats the JSON data for display.
            parsed = json.loads(data)
            print ("Response:")
            print (json.dumps(parsed, sort_keys=True, indent=2))
            conn.close()

        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))

        ####################################

    

        ####################################    
            
    
    def collect_bill_rates(self):
        for user in self.cur:
            bill_rate = float(user[0]["dev_bill_rate"])
            bill_rate_array.append(bill_rate)
    
    def calculate_work_experience(self):
        return 
    
    def calculate_education(self):
        return 
    
    def identify_country(self):
        return 
    
    def identify_job_category(self):
        return 
    
    def make_covariate_matrix(self):
        return 
    
    def causal_analysis(self):
        return 

In [13]:
myObject = UpworkAnalyzer()
myObject.identify_gender()

https://odesk-prod-portraits.s3.amazonaws.com/Users:illymit:PortraitUrl_50?AWSAccessKeyId=1XVAX3FNQZAFC9GJCFR2&Expires=2147483647&Signature=4ST0u9OimggXBHhBp%2FFwT39JZsk%3D&1496853569327478
https://odesk-prod-portraits.s3.amazonaws.com/Users:marwenkhe:PortraitUrl_50?AWSAccessKeyId=1XVAX3FNQZAFC9GJCFR2&Expires=2147483647&Signature=S9CHvfkgKnxn2IeI9y5ifbMrXNM%3D&1500225193414478
https://odesk-prod-portraits.s3.amazonaws.com/Users:bordum:PortraitUrl_50?AWSAccessKeyId=1XVAX3FNQZAFC9GJCFR2&Expires=2147483647&Signature=p0B2veh4XXfA98FICpOyH1hrcrg%3D
{'url':"https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/images_trial/eureka_1.jpeg"}
Response:
[
  {
    "faceAttributes": {
      "accessories": [], 
      "age": 26.7, 
      "blur": {
        "blurLevel": "low", 
        "value": 0.0
      }, 
      "emotion": {
        "anger": 0.0, 
        "contempt": 0.0, 
        "disgust": 0.0, 
        "fear": 0.0, 
        "happiness": 1.0, 
        "neutral": 0.0, 
        "sadn

In [29]:
for user in cur:
    print "Now we're on user number {0}".format(user_count)
    try:
        # Outcome - Bill Rate (Y)

        # Treatment - Gender (D)
        
        # Matrix of Covariates - Work Experience, Education, Country, Job Category (X)
        
        worker_id = user[0]["ciphertext"]
        print worker_id
        first_name = user[0]["dev_first_name"]
        print first_name
        
        gender_by_detector = "none"
        user_count += 1
        

    except:
        print "This one has no detailed info: {0}".format(user_count)
        no_detailed_info_count += 1

print bill_rate_array 

Now we're on user number 0
~019ef3758cf73d289c
Valentina
Now we're on user number 1
~014c24de20d2446d42
Marwen
Now we're on user number 2
~01f5a19ca9880b867f
Bruno
Now we're on user number 3
~0191a24ec4aadf154b
Matt
Now we're on user number 4
~019309dfcb2b02ed10
Orkun
Now we're on user number 5
~01dc5d676329574590
Ganang
Now we're on user number 6
~016e70f671e01d08f5
Yash
Now we're on user number 7
~0193675f0ca4d4c14a
Alex
Now we're on user number 8
~0125bee9e4f17ef78e
Rayhan
Now we're on user number 9
~01326a7a95a824e410
Minas


In [2]:
# Testing CausalModel 

Y = np.array([13.00, 15.00, 16.00, 14.00, 14.50, 16.50, 18.50])
D = np.array([0, 1, 0, 1, 1, 1, 0])
X = np.array([[1, 1], [2, 2], 
              [2, 1], [1, 1], 
              [2, 1], [1, 2],[2, 2]])
causal = CausalModel(Y, D, X)
causal.est_propensity()

print (causal.propensity)

ValueError: Too few control units: N_c < K+1